In [1]:
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from tqdm.autonotebook import tqdm
from pathlib import Path

<ipython-input-1-6565a33895ec>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
prediction_length = 5

In [3]:
total = pd.read_csv("train.csv", index_col=False)
test = pd.read_csv("test.csv", index_col=False)

<ipython-input-3-eccb44559409>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  total = pd.read_csv("train.csv", index_col=False)


In [4]:
total = total[total["Target"]=="ConfirmedCases"]
test = test[test["Target"]=="ConfirmedCases"]

In [5]:
total = total.fillna("")
total["name"] = total["Country_Region"] + "_" + total["Province_State"] + "_" + total["County"]
total.head()

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue,name
0,1,,,Afghanistan,27657145,0.058359,2020-01-23,ConfirmedCases,0,Afghanistan__
2,3,,,Afghanistan,27657145,0.058359,2020-01-24,ConfirmedCases,0,Afghanistan__
4,5,,,Afghanistan,27657145,0.058359,2020-01-25,ConfirmedCases,0,Afghanistan__
6,7,,,Afghanistan,27657145,0.058359,2020-01-26,ConfirmedCases,0,Afghanistan__
8,9,,,Afghanistan,27657145,0.058359,2020-01-27,ConfirmedCases,0,Afghanistan__


In [6]:
country_list = sorted(list(set(total["name"])))
date_list = sorted(list(set(total["Date"])))
data_dic = {"name": country_list}

for date in date_list:
    tmp = total[total["Date"]==date][["name", "Date", "TargetValue"]]
    tmp = tmp.pivot(index="name", columns="Date", values="TargetValue")
    tmp_values = tmp[date].values
    data_dic[date] = tmp_values
new_df = pd.DataFrame(data_dic)
new_df.head()

,name,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10
0,Afghanistan__,0,0,0,0,0,0,0,0,0,...,545,759,758,787,915,582,791,575,542,683
1,Albania__,0,0,0,0,0,0,0,0,0,...,6,21,20,13,15,20,14,17,36,42
2,Algeria__,0,0,0,0,0,0,0,0,0,...,119,113,107,98,104,115,104,111,117,102
3,Andorra__,0,0,0,0,0,0,0,0,0,...,1,79,7,1,0,0,0,0,0,0
4,Angola__,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,3,1,4,17


In [7]:
total_values = new_df.drop("name", axis=1).values
row, col = total_values.shape
for i in range(row):
    tmp = total_values[i]
    for j in range(col):
        if j > 0:
            tmp[j] = tmp[j] + tmp[j - 1]
    total_values[i] = tmp

In [8]:
feature_dic_population = {}
feature_dic_weight = {}
for date in date_list:
    tmp = total[total["Date"]==date][["Date", "name", "Population", "Weight"]]
    population = tmp.pivot(index="name", columns="Date", values="Population")
    weight = tmp.pivot(index="name", columns="Date", values="Weight")
    feature_dic_population[date] = population[date].values
    feature_dic_weight[date] = weight[date].values
feature_df_population = pd.DataFrame(feature_dic_population)
feature_df_weight = pd.DataFrame(feature_dic_weight)
feature_df_population.head()

,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,...,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10
0,27657145,27657145,27657145,27657145,27657145,27657145,27657145,27657145,27657145,27657145,...,27657145,27657145,27657145,27657145,27657145,27657145,27657145,27657145,27657145,27657145
1,2886026,2886026,2886026,2886026,2886026,2886026,2886026,2886026,2886026,2886026,...,2886026,2886026,2886026,2886026,2886026,2886026,2886026,2886026,2886026,2886026
2,40400000,40400000,40400000,40400000,40400000,40400000,40400000,40400000,40400000,40400000,...,40400000,40400000,40400000,40400000,40400000,40400000,40400000,40400000,40400000,40400000
3,78014,78014,78014,78014,78014,78014,78014,78014,78014,78014,...,78014,78014,78014,78014,78014,78014,78014,78014,78014,78014
4,25868000,25868000,25868000,25868000,25868000,25868000,25868000,25868000,25868000,25868000,...,25868000,25868000,25868000,25868000,25868000,25868000,25868000,25868000,25868000,25868000


In [9]:
populations = []
weights = []
for i in range(feature_df_population.shape[0]):
    populations.append(feature_df_population.values[i][0])
    weights.append(feature_df_weight.values[i][0])

In [10]:
def min_max_scale(lst):
    minimum = min(lst)
    maximum = max(lst)
    new = []
    for i in range(len(lst)):
        new.append((lst[i] - minimum) / (maximum - minimum))
    return new

In [11]:
scaled_populations = min_max_scale(populations)
scaled_weights = min_max_scale(weights)
stat_real_features = []
stat_cat_features = []
for i in range(len(scaled_weights)):
    if 0 <= scaled_weights[i] <= 0.33:
#         country with small number of people
        stat_cat_features.append([1])
    elif 0.33 < scaled_weights[i] <= 0.67:
#         country with median number of people
        stat_cat_features.append([2])
    else:
#         country with large number of people
        stat_cat_features.append([3])
    stat_real_features.append([scaled_weights[i]])

In [12]:
from gluonts.dataset.common import load_datasets, ListDataset
from gluonts.dataset.field_names import FieldName
from copy import copy

train_df = new_df.drop(["name"], axis=1)
test_target_values = total_values.copy()
train_target_values = [ts[:-prediction_length] for ts in total_values]
cat_cardinality = [3]

start_date = [pd.Timestamp("2020-01-23", freq='1D') for _ in range(len(new_df))]
train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_STATIC_REAL: static_real,
        FieldName.FEAT_STATIC_CAT: static_cat
    }
    for (target, start, static_real,  static_cat) in zip(train_target_values,
                                         start_date,
                                         stat_real_features,
                                        stat_cat_features)
], freq="D")

test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start,
        FieldName.FEAT_STATIC_REAL: static_real,
        FieldName.FEAT_STATIC_CAT: static_cat
    }
    for (target, start, static_real,  static_cat) in zip(test_target_values,
                                         start_date,
                                        stat_real_features, 
                                        stat_cat_features)
], freq="D")

/home/resifis/miniconda3/lib/python3.9/site-packages/gluonts/json.py:45: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
<ipython-input-12-a4b6a397c40d>:10: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  start_date = [pd.Timestamp("2020-01-23", freq='1D') for _ in range(len(new_df))]


In [13]:
next(iter(train_ds))

/home/resifis/miniconda3/lib/python3.9/site-packages/gluonts/dataset/common.py:323: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(string, freq=freq)
/home/resifis/miniconda3/lib/python3.9/site-packages/gluonts/dataset/common.py:326: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if isinstance(timestamp.freq, Tick):
/home/resifis/miniconda3/lib/python3.9/site-packages/gluonts/dataset/common.py:328: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  timestamp.floor(timestamp.freq), timestamp.freq
/home/resifis/miniconda3/lib/python3.9/site-packages/gluonts/dataset/common.py:327: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  return pd.Timestamp(


{'target': array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        4.0000e+00, 4.0000e+00, 5.0000e+00, 7.0000e+00, 7.0000e+00,
        7.0000e+00, 1.1000e+01, 1.6000e+01, 2.1000e+01, 2.2000e+01,
        2.2000e+01, 2.2000e+01, 2.4000e+01, 2.4000e+01, 4.0000e+01,
        4.0000e+01, 7.4000e+01, 8.4000e+01, 9.4000e+01, 1.1000e+02,
        1.1000e+02, 1.2000e+02, 1.7000e+02, 1.7400e+02, 2.3700e+02,
        2.7300e+02, 2.8100e+02, 2.9900

In [14]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer import Trainer

n = 50
estimator = DeepAREstimator(
    prediction_length=prediction_length,
    freq="D",
    distr_output = NegativeBinomialOutput(),
    use_feat_static_real=True,
#     use_feat_static_cat=True,
#     cardinality=cat_cardinality,
    trainer=Trainer(
        learning_rate=1e-5,
        epochs=n,
        num_batches_per_epoch=50,
        batch_size=32
    )
)

predictor = estimator.train(train_ds)

ModuleNotFoundError: No module named 'gluonts.distribution'